<a href="https://colab.research.google.com/github/wongzw/IS4242_Group_4/blob/4-knn-model/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Attempt without mediapipe

Import Developmental Packages

In [ ]:
import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from imutils import paths
import os
from tqdm import tqdm
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
folder_name = f"/content/drive/MyDrive/4. NUS/Studies/Y4S2/IS4242/IS4242/data"

Mounted at /content/drive


Define function to extract features from images

In [ ]:
def extract_features(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Resize the image to a fixed size (50x50)
    resized = cv2.resize(gray, (50, 50))
    # Flatten the image into a 1D array
    flattened = resized.flatten()
    # Return the flattened array as the feature vector
    return flattened


Load dataset

In [ ]:
image_paths = os.listdir(folder_name)
data = []
labels = []

for i in tqdm(range(len(image_paths))):
    image_path = image_paths[i]
    if image_path.endswith(('.jpg','.png')):
      # Load the image and extract its features
      image = cv2.imread(f"{folder_name}/{image_path}")
      features = extract_features(image)
      # Extract the label from the image filename (e.g. "A.jpg")
      label = re.split(r'[-_]', image_path)[0].lower()
      # Add the feature vector and label to the dataset
      data.append(features)
      labels.append(label)

# Convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)


100%|██████████| 1221/1221 [02:54<00:00,  6.98it/s]


Split the dataset into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split
import joblib

In [ ]:
X = data
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

output_dir = f"/content/drive/MyDrive/4. NUS/Studies/Y4S2/IS4242/IS4242/dump/"
joblib.dump(X, output_dir+'X_knn.pkl')
joblib.dump(y, output_dir+'y_knn.pkl')

joblib.dump(X_train, output_dir+'X_knn_train.pkl')
joblib.dump(y_train, output_dir+'y_knn_train.pkl')

joblib.dump(X_test, output_dir+'X_knn_test.pkl')
joblib.dump(y_test, output_dir+'y_knn_test.pkl')


(976, 2500)
(244, 2500)
(976,)
(244,)


['/content/drive/MyDrive/4. NUS/Studies/Y4S2/IS4242/IS4242/dump/y_knn_test.pkl']

Standardise Data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Train the KNN classifier on the training set

In [ ]:
k = 3 # Choose the number of neighbors
classifier = KNeighborsClassifier(n_neighbors=k)
classifier.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=3)

Test the KNN classifier on the testing set

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

classifiction_report = classification_report(y_test, y_pred)
print("Classification Report:", classifiction_report)

Accuracy: 0.01639344262295082
Classification Report:               precision    recall  f1-score   support

           1       0.00      0.00      0.00         7
          10       0.05      0.10      0.07        10
           2       0.00      0.00      0.00         6
           3       0.09      0.14      0.11         7
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         5
           9       0.00      0.00      0.00         8
           a       0.00      0.00      0.00        12
           b       0.00      0.00      0.00         5
           c       0.00      0.00      0.00         7
           d       0.00      0.00      0.00         5
           e       0.00      0.00      0.00         4
           f       0.00      0.00      0.00         4
           g       0.00     

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Predict the label of a new image
new_image = cv2.imread(f"{folder_name}/a_ZW1.jpg")
new_features = extract_features(new_image)
predicted_label = classifier.predict([new_features])[0]
print("Predicted label:", predicted_label)

Predicted label: c


#### Implementation with Mediapipe

In [ ]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 20.1 MB/s eta 0:00:00


In [ ]:
# For capturing hand coordinates
import cv2
import tensorflow as tf
import mediapipe as mp

In [ ]:
# For processing data
import csv
import os
import numpy as np
import pandas as pd
from tqdm import tqdm 
import re

Generate Dataset of Hand Points

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
folder_name = f"/content/drive/MyDrive/4. NUS/Studies/Y4S2/IS4242/IS4242/data"

Mounted at /content/drive


In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [ ]:
image_paths = os.listdir(folder_name)
data = []
labels = []

In [ ]:
with mp_hands.Hands(static_image_mode =True, max_num_hands = 2, min_detection_confidence=0.5) as hands:
    for i in tqdm(range(len(image_paths))):
        image_path = image_paths[i]
    
        if image_path.endswith(('.jpg','.png')):
        # Load the image and extract its features
            image = cv2.flip(cv2.imread(f"{folder_name}/{image_path}"), 1)
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            try:
                # Extract Hand landmarks
                for hand_landmark in results.multi_hand_landmarks:
                    right_hand = hand_landmark.landmark
                    
                right_hand_row = list(np.array([[landmark.x, landmark.y] for landmark in right_hand]).flatten())

                # Concate rows
                row = right_hand_row

                # Extract the label from the image filename (e.g. "A.jpg")
                label = re.split(r'[-_]', image_path)[0].lower()

                data.append(row)
                labels.append(label)
                print(row, label)
            except Exception as e:
                print(f"Error: {e}")
                pass
        else:
            continue


  0%|          | 4/1221 [00:00<01:16, 15.92it/s]

[0.7142850160598755, 0.5385409593582153, 0.6337572336196899, 0.37149056792259216, 0.48400983214378357, 0.28869298100471497, 0.35504433512687683, 0.32366013526916504, 0.28930792212486267, 0.4018591344356537, 0.5308923721313477, 0.2992878258228302, 0.34949690103530884, 0.3263068199157715, 0.26146838068962097, 0.34709593653678894, 0.18276464939117432, 0.3605175018310547, 0.5462238192558289, 0.43290436267852783, 0.3325423300266266, 0.43398138880729675, 0.2243829071521759, 0.42734313011169434, 0.1449185609817505, 0.43072110414505005, 0.55235356092453, 0.5573891401290894, 0.33064502477645874, 0.5434051156044006, 0.371639609336853, 0.5214931964874268, 0.43975257873535156, 0.5174238085746765, 0.5495056509971619, 0.6638360023498535, 0.3830954134464264, 0.6367799043655396, 0.406872421503067, 0.6019423604011536, 0.4628511667251587, 0.5887867212295532] h
[0.7066187858581543, 0.7655515074729919, 0.6444982290267944, 0.7524808049201965, 0.5367423892021179, 0.6606072783470154, 0.48731717467308044, 0.5

  1%|          | 8/1221 [00:00<01:15, 16.06it/s]

[0.5832924246788025, 0.8387327194213867, 0.43176308274269104, 0.7289754152297974, 0.3292398154735565, 0.5712554454803467, 0.24772587418556213, 0.4671157896518707, 0.15355050563812256, 0.40730154514312744, 0.5997492074966431, 0.45843130350112915, 0.676689088344574, 0.30885136127471924, 0.7284280061721802, 0.22580556571483612, 0.7833566665649414, 0.15424296259880066, 0.7220215201377869, 0.5050361156463623, 0.7268787026405334, 0.461495578289032, 0.6424052715301514, 0.5738406777381897, 0.5835456848144531, 0.6627565026283264, 0.8257991671562195, 0.5688118934631348, 0.7989888787269592, 0.5384464263916016, 0.7042907476425171, 0.6424758434295654, 0.6490522623062134, 0.7186553478240967, 0.9198054671287537, 0.6410956382751465, 0.9139887094497681, 0.5841817855834961, 0.8247435092926025, 0.6383493542671204, 0.7683579921722412, 0.6881232261657715] l
[0.44856539368629456, 0.6785751581192017, 0.3184835910797119, 0.5374352931976318, 0.35849159955978394, 0.37259942293167114, 0.5342126488685608, 0.33304

  1%|          | 12/1221 [00:00<01:15, 16.08it/s]

...

[0.8465006351470947, 0.7605932950973511, 0.768062174320221, 0.7820826768875122, 0.690589427947998, 0.7350314855575562, 0.7297654747962952, 0.6257160305976868, 0.7760530114173889, 0.5392130613327026, 0.4921049177646637, 0.5961683988571167, 0.339875727891922, 0.5135278701782227, 0.24821925163269043, 0.4535362720489502, 0.16781491041183472, 0.4003024399280548, 0.5518599152565002, 0.5330380201339722, 0.4032774865627289, 0.41421642899513245, 0.31390470266342163, 0.32409363985061646, 0.23503869771957397, 0.24853506684303284, 0.641774594783783, 0.5126259922981262, 0.611609160900116, 0.426173597574234, 0.6222331523895264, 0.36395755410194397, 0.620539665222168, 0.31283026933670044, 0.7493246793746948, 0.5232364535331726, 0.757026195526123, 0.4732106924057007, 0.7858330011367798, 0.5020166635513306, 0.8051090240478516, 0.54031902551651] 6
[0.7544986605644226, 0.7815252542495728, 0.65756756067276, 0.8005849123001099, 0.594301164150238, 0.744594931602478, 0.6468452215194702, 0.6490338444709778, 0

100%|█████████▉| 1217/1221 [03:55<00:00, 14.86it/s]

[0.5996417999267578, 0.79306960105896, 0.39319679141044617, 0.6466786861419678, 0.2979464530944824, 0.4737524688243866, 0.2598528265953064, 0.33949291706085205, 0.22542867064476013, 0.22370794415473938, 0.49097514152526855, 0.3898925185203552, 0.3047134280204773, 0.4871383011341095, 0.28275197744369507, 0.5815138220787048, 0.31195154786109924, 0.6181690692901611, 0.6045355200767517, 0.4461739957332611, 0.3742164969444275, 0.5598418116569519, 0.3541839122772217, 0.6337900161743164, 0.3889053463935852, 0.6542877554893494, 0.6909634470939636, 0.5264216661453247, 0.46989426016807556, 0.625795304775238, 0.44811615347862244, 0.6728670597076416, 0.4910524785518646, 0.6736036539077759, 0.7469473481178284, 0.6166607737541199, 0.5841648578643799, 0.6635140180587769, 0.5473973751068115, 0.6840924024581909, 0.5743988752365112, 0.6833668947219849] 10
[0.42484617233276367, 0.7716618776321411, 0.3030908405780792, 0.6230785846710205, 0.2516164779663086, 0.43952590227127075, 0.2902316749095917, 0.29268

100%|██████████| 1221/1221 [03:55<00:00,  5.19it/s]

[0.3290475606918335, 0.8202658891677856, 0.2514601945877075, 0.7222768664360046, 0.23357129096984863, 0.6056721210479736, 0.3109889626502991, 0.5485563278198242, 0.4377374053001404, 0.5583396553993225, 0.45086684823036194, 0.4569613039493561, 0.5146531462669373, 0.31616848707199097, 0.5538402199745178, 0.22722947597503662, 0.5969905257225037, 0.15251633524894714, 0.5563005208969116, 0.5062992572784424, 0.6116736531257629, 0.39410755038261414, 0.5218172073364258, 0.46133726835250854, 0.46169739961624146, 0.5265335440635681, 0.6366919279098511, 0.5878300070762634, 0.7180141806602478, 0.48222434520721436, 0.6083428263664246, 0.5354464054107666, 0.5314185619354248, 0.5934763550758362, 0.6885823011398315, 0.692360520362854, 0.7347344756126404, 0.6133419871330261, 0.6361026763916016, 0.6279107928276062, 0.556399405002594, 0.6586756110191345] d
[0.3993390202522278, 0.7440075874328613, 0.2641908824443817, 0.6138280630111694, 0.31508785486221313, 0.4726116955280304, 0.5463234186172485, 0.449061

In [ ]:
pd.DataFrame(data)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0.714285,0.538541,0.633757,0.371491,0.484010,0.288693,0.355044,0.323660,0.289308,0.401859,...,0.439753,0.517424,0.549506,0.663836,0.383095,0.636780,0.406872,0.601942,0.462851,0.588787
1,0.706619,0.765552,0.644498,0.752481,0.536742,0.660607,0.487317,0.527394,0.422199,0.424137,...,0.632126,0.580606,0.471110,0.458589,0.440720,0.348326,0.439008,0.277368,0.417766,0.214393
2,0.590929,0.899462,0.739604,0.737033,0.798656,0.515775,0.735411,0.349554,0.634991,0.329270,...,0.445729,0.466100,0.332255,0.629419,0.279347,0.443670,0.251201,0.340309,0.212039,0.263191
3,0.564546,0.894767,0.428664,0.730309,0.439319,0.573292,0.458046,0.464387,0.452443,0.368997,...,0.432329,0.712584,0.761282,0.662329,0.521680,0.614555,0.472888,0.684964,0.491638,0.743883
4,0.583292,0.838733,0.431763,0.728975,0.329240,0.571255,0.247726,0.467116,0.153551,0.407302,...,0.649052,0.718655,0.919805,0.641096,0.913989,0.584182,0.824744,0.638349,0.768358,0.688123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1172,0.634050,0.800721,0.448968,0.745843,0.315569,0.658995,0.250711,0.588302,0.192561,0.518246,...,0.544835,0.182937,0.738237,0.439439,0.724908,0.309390,0.690684,0.241492,0.646674,0.203861
1173,0.329048,0.820266,0.251460,0.722277,0.233571,0.605672,0.310989,0.548556,0.437737,0.558340,...,0.531419,0.593476,0.688582,0.692361,0.734734,0.613342,0.636103,0.627911,0.556399,0.658676
1174,0.399339,0.744008,0.264191,0.613828,0.315088,0.472612,0.546323,0.449062,0.729571,0.487421,...,0.617182,0.456296,0.773369,0.496504,0.830068,0.380758,0.770958,0.427723,0.710793,0.487394
1175,0.497821,0.830919,0.360241,0.768595,0.253964,0.671540,0.220052,0.581182,0.320144,0.541033,...,0.722906,0.177029,0.725411,0.545754,0.813127,0.454805,0.865661,0.400159,0.909829,0.345541


In [ ]:
print(labels)

['h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'm', 'n', 'o', 'r', 's', 'u', 'v', 'w', 'x', 'y', '1', '2', '3', '4', '5', '6', '7', '8', '10', 'l', 'p', 'q', 't', '9', 'a', 'b', 'c', 'g', 'e', 'd', 'f', 'h', 'j', 'i', 'l', 'm', 'k', 'o', 'n', 'r', 's', 'u', 'v', 't', 'x', 'y', 'w', 'z', '1', '2', '4', '5', '3', '7', '8', '6', '10', '9', 'a', 'c', 'b', 'd', 'e', 'f', 'g', 'i', 'j', 'h', 'l', 'm', 'k', 'o', 'p', 'q', 'n', 's', 't', 'r', 'v', 'w', 'u', 'y', 'z', 'x', '1', '3', '4', '2', '6', '7', '5', '9', '10', '8', 'a', 'b', 'd', 'c', 'f', 'g', 'e', 'i', 'j', 'h', 'l', 'm', 'n', 'k', 'p', 'q', 'o', 's', 't', 'u', 'r', 'w', 'v', 'x', 'z', 'y', '1', '2', '3', '4', '6', '5', '8', '9', '7', '10', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'n', 'm',

In [ ]:
import joblib

output_dir = f"/content/drive/MyDrive/4. NUS/Studies/Y4S2/IS4242/IS4242/dump/"
joblib.dump(data, output_dir+'data_knn_2.pkl')
joblib.dump(labels, output_dir+'labels_knn_2.pkl')

['/content/drive/MyDrive/4. NUS/Studies/Y4S2/IS4242/IS4242/dump/labels_knn_2.pkl']

Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

# We will take 33% from 1000 for our test data.
# Recommended value 80:20, 67:33, 50:50
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Train Model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

Test Model

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.53      1.00      0.70         8
          10       0.86      0.55      0.67        11
           2       0.18      0.44      0.26         9
           3       0.38      0.75      0.50         4
           4       0.33      0.75      0.46         8
           5       0.78      0.78      0.78         9
           6       0.33      0.06      0.11        16
           7       0.86      0.50      0.63        12
           8       0.58      1.00      0.74         7
           9       0.09      0.14      0.11         7
           a       0.48      0.92      0.63        12
           b       0.47      0.50      0.48        14
           c       0.45      0.38      0.42        13
           d       0.60      0.60      0.60        10
           e       0.31      0.42      0.36        12
           f       0.57      0.33      0.42        12
           g       0.33      0.71      0.45         7
           h       0.83    

Train-Test Excluding Numeric

In [ ]:
from sklearn.model_selection import train_test_split

numeric_list = ['0','1','2','3','4','5','6','7','8','9','10']
data_non_numeric = []
labels_non_numeric = []

for i in range(len(labels)):
    if labels[i] not in numeric_list:
        data_non_numeric.append(data[i])
        labels_non_numeric.append(labels[i])



In [ ]:
# We will take 33% from 1000 for our test data.
# Recommended value 80:20, 67:33, 50:50
X_train_non_numeric, X_test_non_numeric, y_train_non_numeric, y_test_non_numeric = train_test_split(data_non_numeric, labels_non_numeric, test_size=0.33, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train_non_numeric)

X_train_non_numeric = scaler.transform(X_train_non_numeric)
X_test_non_numeric = scaler.transform(X_test_non_numeric)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(X_train_non_numeric, y_train_non_numeric)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
y_pred_non_numeric = classifier.predict(X_test_non_numeric)
print(classification_report(y_test_non_numeric, y_pred_non_numeric))
print(accuracy_score(y_test_non_numeric, y_pred_non_numeric))

              precision    recall  f1-score   support

           a       0.43      1.00      0.60         9
           b       0.67      0.91      0.77        11
           c       0.40      0.55      0.46        11
           d       0.54      0.78      0.64         9
           e       0.38      0.50      0.43        10
           f       0.89      0.53      0.67        15
           g       0.55      0.60      0.57        10
           h       0.70      0.64      0.67        11
           i       0.62      0.45      0.53        11
           j       0.38      0.23      0.29        13
           k       0.64      0.64      0.64        11
           l       0.78      1.00      0.88         7
           m       0.50      0.38      0.43        13
           n       0.22      0.22      0.22         9
           o       0.57      0.40      0.47        10
           p       0.67      0.36      0.47        11
           q       0.29      0.17      0.21        12
           r       0.40    

Playground

In [ ]:
!pip install opencv-contrib-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [ ]:
# Initialize mediapipe hand capture webcam

cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    max_num_hands = 1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()

        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                coords = hand_landmarks.landmark
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                coords = list(np.array([[landmark.x, landmark.y] for landmark in coords]).flatten())
                coords = scaler.transform([coords])
                
                # Alternative for dataset using z coordinates.
                # Z coordinates is not recommended, since you need to adjust your distance from camera.
#                 coords = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in coords]).flatten())
                
                predicted = classifier.predict(coords)

            # Get status box
            cv2.rectangle(image, (0,0), (100, 60), (245, 90, 16), -1)

            # Display Class
            cv2.putText(image, 'CLASS'
                        , (20,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(predicted[0])
                        , (20,45), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('MediaPipe Hands', image)

        # Press esc to close webcam
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()